# Training CALTECH256 over DenseNet121

In [ ]:
import keras
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import Adam
import numpy as np
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

#Transfer Learning of DenseNet121 with pre-trained imagenet weights

In [ ]:
base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False)



x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(257, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Load Dataset with flow_from_directory

#### Dataset: [CALTECH256](http://www.vision.caltech.edu/Image_Datasets/Caltech256/)

#### Before proceeding further divide the CALTECH256 dataset into train and test folders with 80% of images for training set and 20% images for testset from total 30,607

##### Note: Similarly, for CCP augmented training, using the divided training set (i.e., 24485 images), create CCP attacked images of training images and include them in the divided folder(i.e., total 48970 images in training for CCP augmented training)

In [ ]:
TRAINING_DIR = "./Caltech256/train"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "./Caltech256/test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(256,256),
	class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(256,256),
	class_mode='categorical'
)

### Create numpy arrays of Test Dataset (to keep track of same test data)

In [ ]:
all_classes = os.listdir("Caltech_256/test/")

count = 0
test_images = []
test_labels = []

for eachclass in all_classes:
    images = glob.glob('Caltech_256/test/' + eachclass + '/*.jpg')
    for img in images:
        read = cv2.imread(img)
        image = cv2.cvtColor(read, cv2.COLOR_BGR2RGB)
        test_images.append(image)
        test_labels.append(count)
    count += 1
    
    
test_im = np.array(test_images)
test_la = np.array(test_labels)

np.save('test_images_caltech256_fordensenet121.npy',test_im)
np.save('test_labels_caltech256_fordensenet121.npy',test_la)

# Model Compile and Training

### First 20 Epochs

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator, epochs=20, validation_data = validation_generator, verbose = 1)

### Next 20 Epochs

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator, epochs=20, validation_data = validation_generator, verbose = 1)

In [ ]:
model.save("Densenet121_over_Caltech256.h5")

# Evaluate Model

In [ ]:
import glob
import os
import numpy as np
import random


# initalize
check = 0
correct = 0

# Load dataset and labels
ims = np.load('test_images_caltech256_fordensenet121.npy')
lbs = np.load('test_labels_caltech256_fordensenet121.npy')
total = len(ims)

# Count wrongly classified
for ind in range(len(ims)):
    temp = ims[ind]
    image = temp/255.0
    img_pred = np.expand_dims(image, axis=0)
    result = new_model.predict(img_pred)
    cl = np.argmax(result[0])
    check = lbs[ind]
    
    if cl==check:
        correct += 1
        
print(" Accuracy - " + str((correct/total) * 100) + '%')

# Testing on CCP Attack
### Testing the CALTECH test dataset with s=1 and b=30
#### (CCP_F and CCP_V)

In [ ]:
# Load Model
import keras
model = keras.models.load_model('Densenet121_over_Caltech256.h5')

In [ ]:
# CCP ATTACK FUNCTIONS
def change_brightness(image, alpha, beta):
  new_image = np.zeros(image.shape, np.int64)
  new_image = np.clip( alpha*image + beta, 0, 255)
  return new_image
	
def CCP_Attack_Brightness(image, transform):
	img = copy.copy(image)
	for channel in range(img.shape[2]):
		temp1 = image[:,:,0]
		temp2 = image[:,:,1]
		temp3 = image[:,:,2]

		temp = temp1 * transform[channel][0] + temp2 * transform[channel][1] + temp3 * transform[channel][2]

		img[:,:,channel] = temp/3

	img1 = change_brightness(img, 1, 30)
	return img1

In [ ]:
# Test for CCP Attack
import glob
import numpy as np
import random


# initalize
check = 0
correct = 0
total = 0

# Generate weights
a = np.random.uniform(low=0.0, high=1.0, size=(3,))
b = np.random.uniform(low=0.0, high=1.0, size=(3,))
c = np.random.uniform(low=0.0, high=1.0, size=(3,))
transform = np.array([a,b,c])

# Load dataset and labels
ims = np.load('test_images_caltech256_fordensenet121.npy')
lbs = np.load('test_labels_caltech256_fordensenet121.npy')

# Count wrongly classified
for ind in range(len(ims)):

    ## Comment or Uncomment this for Fixed or Variable settings respectively.
    a = np.random.uniform(low=0.0, high=1.0, size=(3,))
    b = np.random.uniform(low=0.0, high=1.0, size=(3,))
    c = np.random.uniform(low=0.0, high=1.0, size=(3,))
    transform = np.array([a,b,c])

    temp = CCP_Attack_Brightness(ims[ind], transform)
    image = temp/255.0
    img_pred = np.expand_dims(image, axis=0)
    result = new_model.predict(img_pred)
    cl = np.argmax(result[0])
    check = lbs[ind]
    
    if cl==check:
        correct += 1

print(" Accuracy - " + str((correct/total) * 100) + '%')